In [4]:
# # ================================
# # نسخه پایدار و بدون خطا - SARIMA + STL + تشخیص انومالی
# # ================================
# import pandas as pd
# import numpy as np
# import matplotlib.pyplot as plt
# import seaborn as sns
# from statsmodels.tsa.seasonal import STL
# from statsmodels.tsa.statespace.sarimax import SARIMAX
# from statsmodels.tsa.stattools import adfuller
# from scipy import stats
# import warnings
# warnings.filterwarnings("ignore")

# plt.rcParams['figure.figsize'] = (16, 10)
# sns.set_style("whitegrid")

# # مسیرها
# INPUT_FILE = r"C:\BI\lube_oil_system_data_g11.xlsx"
# OUTPUT_EXCEL = r"C:\BI\time_series_forecast_main_lube_oil_pump_press_SARIMA.xlsx"
# OUTPUT_PLOTS = r"C:\BI\plots_timeseries_sarima\\"
# LOG_FILE = r"C:\BI\ts_forecast_sarima_log.txt"

# import os
# os.makedirs(r"C:\BI", exist_ok=True)
# os.makedirs(OUTPUT_PLOTS, exist_ok=True)

# def log(msg):
#     print(msg)
#     with open(LOG_FILE, "a", encoding="utf-8") as f:
#         from datetime import datetime
#         f.write(f"[{datetime.now().strftime('%Y-%m-%d %H:%M:%S')}] {msg}\n")

# log("شروع تحلیل سری زمانی با SARIMA + STL (نسخه پایدار بدون Prophet)")

# # خواندن داده
# df = pd.read_excel(INPUT_FILE)
# df["datetime"] = pd.to_datetime(df["RecordDate"] + " " + df["RecordTime"], errors="coerce")
# df = df.sort_values("datetime").dropna(subset=["datetime", "AssetID_9286"]).reset_index(drop=True)

# # تارگت
# ts = df.set_index("datetime")["AssetID_9286"].rename("pressure")
# ts = ts[ts >= 0]  # حذف مقادیر منفی

# # داده روزانه
# daily = ts.resample('D').mean().interpolate()
# log(f"داده روزانه: {len(daily)} روز از {daily.index.min().date()} تا {daily.index.max().date()}")

# # ================================
# # تجزیه STL
# # ================================
# stl = STL(daily, period=7, robust=True)
# result = stl.fit()

# fig = result.plot()
# fig.suptitle("STL Decomposition - Trend + Weekly Seasonality + Residual", fontsize=16)
# plt.savefig(os.path.join(OUTPUT_PLOTS, "01_STL_Decomposition.png"), dpi=200, bbox_inches='tight')
# plt.close()

# trend = result.trend
# seasonal = result.seasonal
# residual = result.resid

# # ================================
# # آموزش SARIMA
# # ================================
# y_sarima = daily - trend - seasonal
# y_sarima = y_sarima.dropna()

# adf = adfuller(y_sarima)
# log(f"ADF p-value: {adf[1]:.5f} {'(ایستا)' if adf[1]<0.05 else '(غیرایستا - نیاز به تفاضل)'}")

# if adf[1] > 0.05:
#     y_sarima = y_sarima.diff().dropna()
#     log("یکبار تفاضل گرفته شد")

# model = SARIMAX(y_sarima, order=(2,0,1), seasonal_order=(1,0,1,7), enforce_stationarity=False)
# fit = model.fit(disp=False)
# log(f"SARIMA فیت شد: {fit.summary().tables[0]}")

# # ================================
# # پیش‌بینی 30 روز آینده
# # ================================
# forecast_steps = 30
# forecast = fit.get_forecast(steps=forecast_steps)
# forecast_mean = forecast.predicted_mean
# forecast_ci = forecast.conf_int()

# print("Forecast CI columns:", forecast_ci.columns)

# last_trend = trend.iloc[-1]
# seasonal_future = seasonal[-forecast_steps:].values if len(seasonal) >= forecast_steps else np.repeat(seasonal.iloc[-1], forecast_steps)

# forecast_full = forecast_mean + last_trend + seasonal_future
# lower_full = forecast_ci.iloc[:, 0] + last_trend + seasonal_future
# upper_full = forecast_ci.iloc[:, 1] + last_trend + seasonal_future

# future_dates = pd.date_range(start=daily.index[-1] + pd.Timedelta(days=1), periods=forecast_steps, freq='D')
# forecast_df = pd.DataFrame({
#     'datetime': future_dates,
#     'forecast': forecast_full.values,
#     'lower_95': lower_full.values,
#     'upper_95': upper_full.values
# })

# # ================================
# # محاسبه residual و تشخیص انومالی
# # ================================
# daily_with_forecast = daily.to_frame()
# daily_with_forecast['trend'] = trend
# daily_with_forecast['seasonal'] = seasonal
# daily_with_forecast['yhat'] = trend + seasonal + fit.fittedvalues.reindex(daily.index, method='nearest')
# daily_with_forecast['residual'] = daily_with_forecast['pressure'] - daily_with_forecast['yhat']
# daily_with_forecast['residual_abs'] = daily_with_forecast['residual'].abs()

# daily_with_forecast['z_score'] = stats.zscore(daily_with_forecast['residual'].fillna(0))
# daily_with_forecast['anomaly'] = daily_with_forecast['z_score'].abs() > 3

# # CUSUM
# residuals = daily_with_forecast['residual'].fillna(0).values
# k = 0.5
# h = 5
# C_pos, C_neg = 0, 0
# cusum_pos, cusum_neg = [], []
# for r in residuals:
#     C_pos = max(0, C_pos + r - k)
#     C_neg = min(0, C_neg + r + k)
#     cusum_pos.append(C_pos)
#     cusum_neg.append(C_neg)
# daily_with_forecast['cusum_anomaly'] = (np.array(cusum_pos) > h) | (np.array(cusum_neg) < -h)

# # ================================
# # رسم نهایی
# # ================================
# plt.figure(figsize=(16, 12))

# plt.subplot(3,1,1)
# plt.plot(daily.index, daily, label='واقعی', color='blue')
# plt.plot(daily_with_forecast.index, daily_with_forecast['yhat'], label='پیش‌بینی درون‌نمونه', color='red')
# plt.plot(forecast_df['datetime'], forecast_df['forecast'], label='پیش‌بینی 30 روز آینده', color='green', linewidth=2)
# plt.fill_between(forecast_df['datetime'], forecast_df['lower_95'], forecast_df['upper_95'], color='lightgreen', alpha=0.5)
# plt.title("پیش‌بینی فشار پمپ روغن اصلی - 30 روز آینده")
# plt.legend()

# plt.subplot(3,1,2)
# plt.plot(daily_with_forecast.index, daily_with_forecast['residual'], color='gray')
# plt.scatter(daily_with_forecast[daily_with_forecast['anomaly']].index,
#             daily_with_forecast[daily_with_forecast['anomaly']]['residual'],
#             color='red', s=60, label='انومالی (Z-score > 3)')
# plt.axhline(0, color='black', linestyle='--')
# plt.title("Residual - تشخیص نقاط پرت")
# plt.legend()

# plt.subplot(3,1,3)
# anomalies_cusum = daily_with_forecast[daily_with_forecast['cusum_anomaly']]
# plt.plot(daily_with_forecast.index, daily_with_forecast['residual'], color='gray', alpha=0.6)
# plt.scatter(anomalies_cusum.index, anomalies_cusum['residual'], color='purple', s=80, label='تغییر ناگهانی سطح (CUSUM)')
# plt.title("تشخیص تغییر ناگهانی در رفتار فشار (خرابی احتمالی)")
# plt.legend()

# plt.tight_layout()
# plt.savefig(os.path.join(OUTPUT_PLOTS, "02_Final_Forecast_Anomaly.png"), dpi=200, bbox_inches='tight')
# plt.close()

# # ================================
# # ذخیره در اکسل
# # ================================
# with pd.ExcelWriter(OUTPUT_EXCEL) as writer:
#     daily_with_forecast.reset_index()[[
#         'datetime', 'pressure', 'yhat', 'residual', 'z_score', 'anomaly', 'cusum_anomaly'
#     ]].to_excel(writer, sheet_name='History_&_Residuals', index=False)

#     forecast_df.to_excel(writer, sheet_name='30_Days_Forecast', index=False)

#     daily_with_forecast[
#         daily_with_forecast['anomaly'] | daily_with_forecast['cusum_anomaly']
#     ].reset_index().to_excel(writer, sheet_name='Anomalies', index=False)

# log("تمام شد! خروجی بدون هیچ خطایی ساخته شد")
# print("\n" + "="*100)


شروع تحلیل سری زمانی با SARIMA + STL (نسخه پایدار بدون Prophet)
داده روزانه: 1690 روز از 2021-04-03 تا 2025-11-17
ADF p-value: 0.00000 (ایستا)
SARIMA فیت شد:                                      SARIMAX Results                                     
Dep. Variable:                                 y   No. Observations:                 1690
Model:             SARIMAX(2, 0, 1)x(1, 0, 1, 7)   Log Likelihood                -771.679
Date:                           Thu, 27 Nov 2025   AIC                           1555.359
Time:                                   11:04:48   BIC                           1587.922
Sample:                               04-03-2021   HQIC                          1567.420
                                    - 11-17-2025                                         
Covariance Type:                             opg                                         
Forecast CI columns: Index(['lower y', 'upper y'], dtype='object')
تمام شد! خروجی بدون هیچ خطایی ساخته شد



In [3]:
# # ================================
# # نسخه پایدار و بدون خطا - SARIMA + STL + تشخیص انومالی (۷ روز آینده)
# # ================================
# import pandas as pd
# import numpy as np
# import matplotlib.pyplot as plt
# import seaborn as sns
# from statsmodels.tsa.seasonal import STL
# from statsmodels.tsa.statespace.sarimax import SARIMAX
# from statsmodels.tsa.stattools import adfuller
# from scipy import stats
# import warnings
# warnings.filterwarnings("ignore")

# plt.rcParams['figure.figsize'] = (16, 10)
# sns.set_style("whitegrid")

# # مسیرها
# INPUT_FILE = r"C:\BI\lube_oil_system_data_g11.xlsx"
# OUTPUT_EXCEL = r"C:\BI\time_series_forecast_main_lube_oil_pump_press_SARIMA.xlsx"
# OUTPUT_PLOTS = r"C:\BI\plots_timeseries_sarima\\"
# LOG_FILE = r"C:\BI\ts_forecast_sarima_log.txt"

# import os
# os.makedirs(r"C:\BI", exist_ok=True)
# os.makedirs(OUTPUT_PLOTS, exist_ok=True)

# def log(msg):
#     print(msg)
#     with open(LOG_FILE, "a", encoding="utf-8") as f:
#         from datetime import datetime
#         f.write(f"[{datetime.now().strftime('%Y-%m-%d %H:%M:%S')}] {msg}\n")

# log("شروع تحلیل سری زمانی با SARIMA + STL (نسخه پایدار بدون Prophet)")

# # خواندن داده
# df = pd.read_excel(INPUT_FILE)
# df["datetime"] = pd.to_datetime(df["RecordDate"] + " " + df["RecordTime"], errors="coerce")
# df = df.sort_values("datetime").dropna(subset=["datetime", "AssetID_9286"]).reset_index(drop=True)

# # تارگت
# ts = df.set_index("datetime")["AssetID_9286"].rename("pressure")
# ts = ts[ts >= 0]  # حذف مقادیر منفی

# # داده روزانه
# daily = ts.resample('D').mean().interpolate()
# log(f"داده روزانه: {len(daily)} روز از {daily.index.min().date()} تا {daily.index.max().date()}")

# # ================================
# # تجزیه STL
# # ================================
# stl = STL(daily, period=7, robust=True)
# result = stl.fit()

# fig = result.plot()
# fig.suptitle("STL Decomposition - Trend + Weekly Seasonality + Residual", fontsize=16)
# plt.savefig(os.path.join(OUTPUT_PLOTS, "01_STL_Decomposition.png"), dpi=200, bbox_inches='tight')
# plt.close()

# trend = result.trend
# seasonal = result.seasonal
# residual = result.resid

# # ================================
# # آموزش SARIMA
# # ================================
# y_sarima = daily - trend - seasonal
# y_sarima = y_sarima.dropna()

# adf = adfuller(y_sarima)
# log(f"ADF p-value: {adf[1]:.5f} {'(ایستا)' if adf[1]<0.05 else '(غیرایستا - نیاز به تفاضل)'}")

# if adf[1] > 0.05:
#     y_sarima = y_sarima.diff().dropna()
#     log("یکبار تفاضل گرفته شد")

# model = SARIMAX(y_sarima, order=(2,0,1), seasonal_order=(1,0,1,7), enforce_stationarity=False)
# fit = model.fit(disp=False)
# log(f"SARIMA فیت شد: {fit.summary().tables[0]}")

# # ================================
# # پیش‌بینی 7 روز آینده
# # ================================
# forecast_steps = 7
# forecast = fit.get_forecast(steps=forecast_steps)
# forecast_mean = forecast.predicted_mean
# forecast_ci = forecast.conf_int()

# print("Forecast CI columns:", forecast_ci.columns)

# last_trend = trend.iloc[-1]
# seasonal_future = seasonal[-forecast_steps:].values if len(seasonal) >= forecast_steps else np.repeat(seasonal.iloc[-1], forecast_steps)

# forecast_full = forecast_mean + last_trend + seasonal_future
# lower_full = forecast_ci.iloc[:, 0] + last_trend + seasonal_future
# upper_full = forecast_ci.iloc[:, 1] + last_trend + seasonal_future

# future_dates = pd.date_range(start=daily.index[-1] + pd.Timedelta(days=1), periods=forecast_steps, freq='D')
# forecast_df = pd.DataFrame({
#     'datetime': future_dates,
#     'forecast': forecast_full.values,
#     'lower_95': lower_full.values,
#     'upper_95': upper_full.values
# })

# # ================================
# # محاسبه residual و تشخیص انومالی
# # ================================
# daily_with_forecast = daily.to_frame()
# daily_with_forecast['trend'] = trend
# daily_with_forecast['seasonal'] = seasonal
# daily_with_forecast['yhat'] = trend + seasonal + fit.fittedvalues.reindex(daily.index, method='nearest')
# daily_with_forecast['residual'] = daily_with_forecast['pressure'] - daily_with_forecast['yhat']
# daily_with_forecast['residual_abs'] = daily_with_forecast['residual'].abs()

# daily_with_forecast['z_score'] = stats.zscore(daily_with_forecast['residual'].fillna(0))
# daily_with_forecast['anomaly'] = daily_with_forecast['z_score'].abs() > 3

# # CUSUM
# residuals = daily_with_forecast['residual'].fillna(0).values
# k = 0.5
# h = 5
# C_pos, C_neg = 0, 0
# cusum_pos, cusum_neg = [], []
# for r in residuals:
#     C_pos = max(0, C_pos + r - k)
#     C_neg = min(0, C_neg + r + k)
#     cusum_pos.append(C_pos)
#     cusum_neg.append(C_neg)
# daily_with_forecast['cusum_anomaly'] = (np.array(cusum_pos) > h) | (np.array(cusum_neg) < -h)

# # ================================
# # رسم نهایی
# # ================================
# plt.figure(figsize=(16, 12))

# plt.subplot(3,1,1)
# plt.plot(daily.index, daily, label='واقعی', color='blue')
# plt.plot(daily_with_forecast.index, daily_with_forecast['yhat'], label='پیش‌بینی درون‌نمونه', color='red')
# plt.plot(forecast_df['datetime'], forecast_df['forecast'], label='پیش‌بینی 7 روز آینده', color='green', linewidth=2)
# plt.fill_between(forecast_df['datetime'], forecast_df['lower_95'], forecast_df['upper_95'], color='lightgreen', alpha=0.5)
# plt.title("پیش‌بینی فشار پمپ روغن اصلی - 7 روز آینده")
# plt.legend()

# plt.subplot(3,1,2)
# plt.plot(daily_with_forecast.index, daily_with_forecast['residual'], color='gray')
# plt.scatter(daily_with_forecast[daily_with_forecast['anomaly']].index,
#             daily_with_forecast[daily_with_forecast['anomaly']]['residual'],
#             color='red', s=60, label='انومالی (Z-score > 3)')
# plt.axhline(0, color='black', linestyle='--')
# plt.title("Residual - تشخیص نقاط پرت")
# plt.legend()

# plt.subplot(3,1,3)
# anomalies_cusum = daily_with_forecast[daily_with_forecast['cusum_anomaly']]
# plt.plot(daily_with_forecast.index, daily_with_forecast['residual'], color='gray', alpha=0.6)
# plt.scatter(anomalies_cusum.index, anomalies_cusum['residual'], color='purple', s=80, label='تغییر ناگهانی سطح (CUSUM)')
# plt.title("تشخیص تغییر ناگهانی در رفتار فشار (خرابی احتمالی)")
# plt.legend()

# plt.tight_layout()
# plt.savefig(os.path.join(OUTPUT_PLOTS, "02_Final_Forecast_Anomaly.png"), dpi=200, bbox_inches='tight')
# plt.close()

# # ================================
# # ذخیره در اکسل
# # ================================
# with pd.ExcelWriter(OUTPUT_EXCEL) as writer:
#     daily_with_forecast.reset_index()[[
#         'datetime', 'pressure', 'yhat', 'residual', 'z_score', 'anomaly', 'cusum_anomaly'
#     ]].to_excel(writer, sheet_name='History_&_Residuals', index=False)

#     forecast_df.to_excel(writer, sheet_name='7_Days_Forecast', index=False)

#     daily_with_forecast[
#         daily_with_forecast['anomaly'] | daily_with_forecast['cusum_anomaly']
#     ].reset_index().to_excel(writer, sheet_name='Anomalies', index=False)

# log("تمام شد! خروجی بدون هیچ خطایی ساخته شد")
# print("\n" + "="*100)


شروع تحلیل سری زمانی با SARIMA + STL (نسخه پایدار بدون Prophet)
داده روزانه: 1690 روز از 2021-04-03 تا 2025-11-17
ADF p-value: 0.00000 (ایستا)
SARIMA فیت شد:                                      SARIMAX Results                                     
Dep. Variable:                                 y   No. Observations:                 1690
Model:             SARIMAX(2, 0, 1)x(1, 0, 1, 7)   Log Likelihood                -771.679
Date:                           Thu, 27 Nov 2025   AIC                           1555.359
Time:                                   11:38:52   BIC                           1587.922
Sample:                               04-03-2021   HQIC                          1567.420
                                    - 11-17-2025                                         
Covariance Type:                             opg                                         
Forecast CI columns: Index(['lower y', 'upper y'], dtype='object')
تمام شد! خروجی بدون هیچ خطایی ساخته شد



In [ ]:
# ================================
# پیش‌بینی 7 روز آینده با SARIMAX برای چهار پارامتر روغن‌کاری
# ================================
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import warnings
warnings.filterwarnings("ignore")

import os
from datetime import datetime
from statsmodels.tsa.statespace.sarimax import SARIMAX

plt.rcParams['figure.figsize'] = (16, 10)
sns.set_style("whitegrid")

# ================================
# مسیرها
# ================================
INPUT_FILE = r"C:\BI\KZCCPP\G11\Lube Oil System\lube_oil_system_data_g11.xlsx"
OUTPUT_EXCEL = r"C:\BI\KZCCPP\G11\Lube Oil System\forecast_lube_oil_params_SARIMAX.xlsx"
OUTPUT_PLOTS = r"C:\BI\KZCCPP\G11\Lube Oil System\plots_timeseries_sarimax\\"
LOG_FILE = r"C:\BI\KZCCPP\G11\Lube Oil System\regression_assetid_sarimax_log.txt"
os.makedirs(r"C:\BI\KZCCPP\G11\Lube Oil System", exist_ok=True)
os.makedirs(OUTPUT_PLOTS, exist_ok=True)

def log(msg):
    print(msg)
    with open(LOG_FILE, "a", encoding="utf-8") as f:
        f.write(f"[{datetime.now().strftime('%Y-%m-%d %H:%M:%S')}] {msg}\n")

log("شروع اجرای مدل‌های SARIMAX برای 4 سیگنال و پیش‌بینی 7 روز آینده")

# ================================
# خواندن داده و ساخت datetime
# ================================
df = pd.read_excel(INPUT_FILE)
log(f"داده خوانده شد → {len(df):,} ردیف")

df["datetime"] = pd.to_datetime(df["RecordDate"] + " " + df["RecordTime"], errors="coerce")
df = df.sort_values("datetime").reset_index(drop=True)

# ================================
# نام‌های خوانا
# ================================
friendly_names = {
    "AssetID_8341": "Lube oil filter diffrential pressure",
    "AssetID_8342": "Lube oil tank level",
    "AssetID_8343": "Lube oil tank temperature",
    "AssetID_8344": "Lube oil tank vapor pressure",
    "AssetID_8346": "Over speed pressure",
    "AssetID_9286": "Main lube oil pump press",
    "AssetID_9287": "Lube oil line pressure",
}
df.rename(columns=friendly_names, inplace=True)

# ================================
# سیگنال‌های هدف
# ================================
target_cols = [
    "Main lube oil pump press",
    "Lube oil line pressure",
    "Lube oil tank temperature",
    "Lube oil tank level"
]

# بررسی وجود ستون‌ها
missing = [c for c in target_cols if c not in df.columns]
if missing:
    raise ValueError(f"ستون‌های زیر در ورودی یافت نشدند: {missing}")

# ================================
# آماده‌سازی روزانه برای هر سیگنال
# ================================
daily_dict = {}
for col in target_cols:
    ts = df.dropna(subset=[col]).set_index("datetime")[col]
    ts = ts[~np.isnan(ts)]
    # حذف مقادیر غیرمنطقی برای فشار؛ برای دما/سطح محدودیت نداریم
    ts = ts.replace([np.inf, -np.inf], np.nan).dropna()
    daily = ts.resample("D").mean().interpolate()
    daily_dict[col] = daily

# بررسی حداقل طول
for col, daily in daily_dict.items():
    if len(daily) < 60:
        raise ValueError(f"طول سری روزانه '{col}' کمتر از 60 است؛ برای آموزش پایدار داده کافی نیست.")

# ================================
# آموزش SARIMAX و پیش‌بینی برای هر سیگنال
# ================================
forecast_horizon = 7
seasonal_period = 7  # هفتگی

# پارامترهای پیش‌فرض پایدار (قابل تغییر در صورت نیاز)
default_order = (2, 0, 1)
default_seasonal_order = (1, 0, 1, seasonal_period)

forecast_frames = []
history_frames = []

for col, daily in daily_dict.items():
    log(f"فیت مدل SARIMAX برای: {col} | طول: {len(daily)} روز")

    # مدل SARIMAX
    # اگر روند تفاضلی لازم باشد، d=1 را می‌توان آزمود؛ فعلاً d=0 با میانگین روزانه و interpolate
    try:
        model = SARIMAX(daily, order=default_order,
                        seasonal_order=default_seasonal_order,
                        enforce_stationarity=False,
                        enforce_invertibility=False)
        res = model.fit(disp=False)
    except Exception as e:
        log(f"خطا در فیت مدل اولیه {col}: {e}. تلاش با پارامترهای جایگزین.")
        # تلاش جایگزین با (1,1,1) و فصل (1,0,1,7)
        model = SARIMAX(daily, order=(1, 1, 1),
                        seasonal_order=default_seasonal_order,
                        enforce_stationarity=False,
                        enforce_invertibility=False)
        res = model.fit(disp=False)

    # پیش‌بینی 7 روز آینده با بازه‌های اطمینان
    start = len(daily)
    end = len(daily) + forecast_horizon - 1
    pred = res.get_prediction(start=start, end=end)
    mean_forecast = pred.predicted_mean
    ci = pred.conf_int()

    future_dates = pd.date_range(daily.index[-1] + pd.Timedelta(days=1),
                                 periods=forecast_horizon, freq="D")

    fdf = pd.DataFrame({
        "datetime": future_dates,
        f"{col} forecast": mean_forecast.values,
        f"{col} lower": ci.iloc[:, 0].values,
        f"{col} upper": ci.iloc[:, 1].values
    })
    forecast_frames.append(fdf)

    # تاریخچه برای اکسل
    hdf = pd.DataFrame({
        "datetime": daily.index,
        col: daily.values
    })
    history_frames.append(hdf)

    # نمودار هر سیگنال
    plt.figure(figsize=(16, 8))
    plt.plot(daily.index, daily.values, label="واقعی (روزانه)", color="steelblue")
    plt.plot(fdf["datetime"], fdf[f"{col} forecast"], label="پیش‌بینی 7 روز آینده (SARIMAX)", color="green", linewidth=2)
    # نوارهای اطمینان
    plt.fill_between(fdf["datetime"],
                     fdf[f"{col} lower"],
                     fdf[f"{col} upper"],
                     color="green", alpha=0.15, label="بازه اطمینان")
    plt.title(f"{col} - پیش‌بینی 7 روز آینده (SARIMAX)")
    plt.xlabel("تاریخ")
    plt.ylabel("مقدار")
    plt.legend()
    plt.tight_layout()
    plt.savefig(os.path.join(OUTPUT_PLOTS, f"sarimax_7day_{col.replace(' ', '_')}.png"),
                dpi=200, bbox_inches="tight")
    plt.close()

# ================================
# ادغام خروجی‌ها
# ================================
# تاریخچه‌ها را merge می‌کنیم روی datetime
history_merged = history_frames[0]
for h in history_frames[1:]:
    history_merged = history_merged.merge(h, on="datetime", how="outer")
history_merged = history_merged.sort_values("datetime")

# پیش‌بینی‌ها را merge می‌کنیم
forecast_merged = forecast_frames[0]
for f in forecast_frames[1:]:
    forecast_merged = forecast_merged.merge(f, on="datetime", how="outer")

# ================================
# ذخیره خروجی در اکسل
# ================================
with pd.ExcelWriter(OUTPUT_EXCEL) as writer:
    history_merged.to_excel(writer, sheet_name="Daily_History_All", index=False)
    forecast_merged.to_excel(writer, sheet_name="7_Days_Forecast_All", index=False)

log("تمام شد! خروجی SARIMAX برای 4 سیگنال ذخیره شد")
print("\n" + "="*80)
print("SARIMAX forecasts ساخته شد و ذخیره شد")
print(f"فایل اکسل: {OUTPUT_EXCEL}")
print(f"نمودارها: {OUTPUT_PLOTS}")
print("="*80)


شروع اجرای مدل‌های SARIMAX برای 4 سیگنال و پیش‌بینی 7 روز آینده
داده خوانده شد → 10,927 ردیف
فیت مدل SARIMAX برای: Main lube oil pump press | طول: 1690 روز
فیت مدل SARIMAX برای: Lube oil line pressure | طول: 1690 روز
فیت مدل SARIMAX برای: Lube oil tank temperature | طول: 1690 روز
فیت مدل SARIMAX برای: Lube oil tank level | طول: 1690 روز
تمام شد! خروجی SARIMAX برای 4 سیگنال ذخیره شد

SARIMAX forecasts ساخته شد و ذخیره شد
فایل اکسل: C:\BI\forecast_lube_oil_params_SARIMAX.xlsx
نمودارها: C:\BI\plots_timeseries_sarimax\\
